# Import required libraries

In [2]:
!pip install wget # to download data
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install gensim
!pip install nltk
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
/bin/bash: line 1: python: command not found
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
%matplotlib inline
import numpy as np
import gensim
import matplotlib.pyplot as plt
import seaborn as sns
import wget
import spacy
from spacy.tokenizer import Tokenizer
from gensim.models import Word2Vec
from sklearn.model_selection import GridSearchCV
import scipy.stats
import tqdm

from tqdm import tqdm
from nltk.corpus import stopwords
import nltk
import re
from collections import defaultdict
import jsonlines

import zipfile

# Implicit representations

1. Create an embedding for the words using the (cleaned) Mosaico dataset by fitting a model.
2. Replace WordNet senses for each word and then create embeddings as before.

We can use different models to create word/sense embeddings and log which one performs better.

### Load the dictionary

In [3]:
# Take all the lines in the file
lines = []
with jsonlines.open('data/sample_annotated_sentences/500000.jsonl') as reader:

    limit = 10
    for i, line in enumerate(reader):

        if i < limit:
            print(f'{i:3d} | {line}')

        lines.append(line)

  0 | {'text': 'In Hinduism , the 60th birthday of a man is called Sashti poorthi .', 'annotations': [{'token_span': [5, 6], 'label': 'birthday%1:28:00::'}, {'token_span': [8, 9], 'label': 'man%1:18:00::'}]}
  1 | {'text': 'In New Zealand , both Metal Box and Second Edition briefly entered the Top 50 Albums Chart .', 'annotations': [{'token_span': [6, 7], 'label': 'box%1:06:00::'}, {'token_span': [8, 9], 'label': 'second%5:00:00:ordinal:00'}, {'token_span': [11, 12], 'label': 'enter%2:33:00::'}, {'token_span': [13, 14], 'label': 'top%3:00:00::'}]}
  2 | {'text': 'Article with extensive bibliography Article about the book " L\'hymnaire d\'Adonis " of Jacques d\'Adelswärd - Fersen ( in Spanish ) Pictures of Villa Lysis today [ 1 ] [ 2 ]', 'annotations': [{'token_span': [0, 1], 'label': 'article%1:10:00::'}, {'token_span': [4, 5], 'label': 'article%1:10:00::'}, {'token_span': [7, 8], 'label': 'book%1:10:00::'}]}
  3 | {'text': 'The root cluster attached to the basal plate of the bulb is t

In [4]:
def pretty_print(line):
    for k, v in line.items():
        print(f'{k}: ', end='')
        if isinstance(v, list) and len(v) > 0:
            print(v[0])
            for i in range(1, len(v)):
                print(f"{' '*(len(k) + 2)}{v[i]}")
        else:
            print(v)

In [5]:
# This is the structure of the data so far
sample_line = lines[0].copy()

pretty_print(sample_line)

text: In Hinduism , the 60th birthday of a man is called Sashti poorthi .
annotations: {'token_span': [5, 6], 'label': 'birthday%1:28:00::'}
             {'token_span': [8, 9], 'label': 'man%1:18:00::'}


In [6]:
# Download stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Load tje pretrained spaCy model
nlp = spacy.load("en_core_web_sm")

# Regex to discard numbers, special characters and punctuation
regexp_alphbetic = re.compile('[^a-zA-Z]+')

[nltk_data] Downloading package stopwords to /home/daniel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
from spacy.lang.en.examples import sentences 

doc = nlp(sentences[0])
print(doc.text)
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.dep_)

Apple is looking at buying U.K. startup for $1 billion
Apple Apple PROPN nsubj
is be AUX aux
looking look VERB ROOT
at at ADP prep
buying buy VERB pcomp
U.K. U.K. PROPN dobj
startup startup NOUN dep
for for ADP prep
$ $ SYM quantmod
1 1 NUM compound
billion billion NUM pobj


There are sentences that are tokenized differently with respect to a simple split. We can't use token_span if we tokenize with the spaCy model. In some cases this is due to hyphens (`co-authored`), in other cases this is due to apostrophes (`Johnson's`) or to something like `.[` that is split into `.` and `[` from the spacy tokenizer and into a single token from a simple string.split() 

In [8]:
def count_different_tokenization_indexes(lines, model, print_lines=False):

    count = 0
    for line in lines:

        text = line['text']
        annotations = line['annotations']

        tokens = text.split()

        doc = model(text)
        tokens_doc = [token.text for token in doc]

        if len(tokens) != len(tokens_doc):
            count += 1

            if print_lines:
                print(text)
                print(f'{tokens}')
                print(f'{tokens_doc}')
                for annotation in annotations:
                    print(annotation)
                print()

    return count

test_limit = 1000
count = count_different_tokenization_indexes(lines[:test_limit], nlp, print_lines=True)
print(f'{count}/{test_limit} sentences are tokenized differently')

The new world of English words came out in 1658 and a dictionary of 40,000 words had been prepared in 1721 by Nathan Bailey , though none was as comprehensive in breadth or style as Johnson's .
['The', 'new', 'world', 'of', 'English', 'words', 'came', 'out', 'in', '1658', 'and', 'a', 'dictionary', 'of', '40,000', 'words', 'had', 'been', 'prepared', 'in', '1721', 'by', 'Nathan', 'Bailey', ',', 'though', 'none', 'was', 'as', 'comprehensive', 'in', 'breadth', 'or', 'style', 'as', "Johnson's", '.']
['The', 'new', 'world', 'of', 'English', 'words', 'came', 'out', 'in', '1658', 'and', 'a', 'dictionary', 'of', '40,000', 'words', 'had', 'been', 'prepared', 'in', '1721', 'by', 'Nathan', 'Bailey', ',', 'though', 'none', 'was', 'as', 'comprehensive', 'in', 'breadth', 'or', 'style', 'as', 'Johnson', "'s", '.']
{'token_span': [1, 2], 'label': 'world%1:14:01::'}
{'token_span': [4, 5], 'label': 'word%1:10:00::'}
{'token_span': [11, 12], 'label': 'dictionary%1:10:00::'}
{'token_span': [14, 15], 'label

Hyphens (-) are used to join two or more words that act as a single term, to form some compound word. Looping through the tokens produced by the model as it is we will end up with two separate tokens for intra-hyphen words, and this could solve the problem as we can consider both, but the position of the senses in the annotations will differ from the original one. Looping through the tokens produced by a split we will end up with a single token for intra-hyphen words and this won't be correctly lemmatized by the spacy model unless we implement some complex logic to account for hyphens. We could split the text with respect to spaces and dashes but even in that case the index of the annotated words will differ:

In [9]:
dashed_text = "laptop-cover co-authored well-known stay-at-home 40.000 L'hymnaire"

# Define the splitting pattern to match spaces and dashes
split_pattern = re.compile(r'[ -]')

# Split the string using the pattern
result = re.split(split_pattern, dashed_text)

# Filter out empty strings resulting from consecutive spaces or dashes
result = [s for s in result if s]

print(result)


['laptop', 'cover', 'co', 'authored', 'well', 'known', 'stay', 'at', 'home', '40.000', "L'hymnaire"]


We can define a [custom tokenizer](https://stackoverflow.com/questions/55241927/spacy-intra-word-hyphens-how-to-treat-them-one-word) for the model that treats this as a single token:

In [10]:
def custom_tokenizer(nlp):
    infix_re = re.compile(r'(\w+)-(\w+)')  # Define an infix regex pattern for hyphens between words

    return Tokenizer(nlp.vocab, infix_finditer=infix_re.finditer)

custom_nlp = spacy.load("en_core_web_sm")
custom_nlp.tokenizer = custom_tokenizer(custom_nlp)

custom_doc = custom_nlp(dashed_text)
doc = nlp(dashed_text)

print([t.lemma_ for t in custom_doc])
print([t.lemma_ for t in doc])


['laptop-cover', 'co-authore', 'well-known', 'stay-at-home', '40.000', "l'hymnaire"]
['laptop', '-', 'cover', 'co', '-', 'author', 'well', '-', 'know', 'stay', '-', 'at', '-', 'home', '40.000', "l'hymnaire"]


In [11]:
count_custom_nlp = count_different_tokenization_indexes(lines[:test_limit], custom_nlp, print_lines=True)
print(f'{count_custom_nlp}/{test_limit} sentences are tokenized differently')

0/1000 sentences are tokenized differently


In [12]:
def preprocess_line(line, 
                    model,
                    stopwords=None, 
                    lemmatize=True,
                    punctuation=None,
                    filtering_regex=None, 
                    replace_with_lemma_key=True,
                    produce_both=False
                    ):


    if stopwords is None:
        stopwords = []

    if punctuation is None:
        punctuation = []

    # Take text and annotations from the line
    text = line['text']
    annotations = line['annotations']

    # Indexes of the words we may want to replace with a lemma key
    # {token1_index: token1, ...}
    tokens = text.split()
    annotation_indexes = {annotation['token_span'][0]: (tokens[annotation['token_span'][0]], annotation['label']) for annotation in annotations}

    doc = model(text)

    # Just in case
    # tokens_doc = [token.text for token in doc]
    # assert len(tokens_doc) == len(tokens)

    # This will be then joined to create a new preprocessed sentence
    new_tokens = []
    new_tokens_sense = []
    
    # For each token, check if it is a stopword, punctuation, if it matches the filter regex 
    # and if it needs to be retained or replaced with its lemma key
    for i, token in enumerate(doc):

        token_text = token.lemma_ if lemmatize else token
        token_text = token_text.lower()
        
        # The token is a word we can replace with the lemma key
        if i in annotation_indexes.keys():

            # We may want to produce both sentences, one with lemma key and one without
            # in order to train a model for word embeddings and another model with
            # sense embeddings
            if produce_both:
                new_tokens.append(token_text)
                new_tokens_sense.append(annotation_indexes[i][1])
            else:
                if replace_with_lemma_key:
                    new_tokens.append(annotation_indexes[i][1])
                else:
                    new_tokens.append(token_text)
                
        # The token is a stopword
        elif token_text in stopwords:
            continue

        # The token is punctuation
        elif token_text in punctuation:
            continue

        # The token matches the regex, it is not valid
        elif filtering_regex is not None and filtering_regex.search(token_text):
            continue

        # The token is valid, we can retain it
        else:
            new_tokens.append(token_text)
            if produce_both:
                new_tokens_sense.append(token_text)

    return ' '.join(new_tokens), ' '.join(new_tokens_sense)


In [13]:
sample_line_2 = {
    'text': "The new world of English words came out in 1658 and a dictionary of 40,000 words had been prepared in 1721 by Nathan Bailey , though none was as comprehensive in breadth or style as Johnson's .",
    'annotations': [
        {'token_span': [1, 2], 'label': 'world%1:14:01::'},
        {'token_span': [4, 5], 'label': 'word%1:10:00::'},
        {'token_span': [11, 12], 'label': 'dictionary%1:10:00::'},
        {'token_span': [14, 15], 'label': 'word%1:10:00::'},
        {'token_span': [32, 33], 'label': 'style%1:10:00::'}
    ]
}

sample_line_3 = {
    'text': "Even without the availability of either co-receptor ( even CCR5 ) , the virus can still invade cells if gp41 were to go through an alteration ( including its cytoplasmic tail ) that resulted in the independence of CD4 without the need of CCR5 and / or CXCR4 as a doorway .",
    'annotations': [
        {'token_span': [17, 18], 'label': 'cell%1:03:00::'},
        {'token_span': [30, 31], 'label': 'tail%1:05:00::'},
        {'token_span': [50, 51], 'label': 'doorway%1:06:00::'}
    ]
}

print('Data before: ')
print(sample_line_3['text'])
print('Preprocessed data:')
print(preprocess_line(sample_line_3, custom_nlp, stopwords=stop_words, filtering_regex=regexp_alphbetic))

Data before: 
Even without the availability of either co-receptor ( even CCR5 ) , the virus can still invade cells if gp41 were to go through an alteration ( including its cytoplasmic tail ) that resulted in the independence of CD4 without the need of CCR5 and / or CXCR4 as a doorway .
Preprocessed data:
('even without availability either even virus still invade cell%1:03:00:: go alteration include cytoplasmic tail%1:05:00:: result independence without need doorway%1:06:00::', '')


In [14]:
print('Data:')
pretty_print(sample_line)
print('-' * (len(sample_line['text']) + len('text: ')))

sample_stopwords = ['in', 'the', 'of', 'a']
sample_punctuation = [',', '.']

print('Preprocessed data:')
print(preprocess_line(sample_line, nlp, stopwords=sample_stopwords, punctuation=sample_punctuation))

Data:
text: In Hinduism , the 60th birthday of a man is called Sashti poorthi .
annotations: {'token_span': [5, 6], 'label': 'birthday%1:28:00::'}
             {'token_span': [8, 9], 'label': 'man%1:18:00::'}
-------------------------------------------------------------------------
Preprocessed data:
('hinduism 60th birthday%1:28:00:: man%1:18:00:: be call sashti poorthi', '')


In [15]:
# Test the above script with spaCy stopwords, a regex to filter out words 
# if they contain punctuation, special characters and numbers but without
# replacing lemma keys

print('Data:')
pretty_print(sample_line)
print('-' * (len(sample_line['text']) + len('text: ')))

print('Preprocessed data:')
print(preprocess_line(sample_line, nlp, stopwords=stop_words, filtering_regex=regexp_alphbetic))

Data:
text: In Hinduism , the 60th birthday of a man is called Sashti poorthi .
annotations: {'token_span': [5, 6], 'label': 'birthday%1:28:00::'}
             {'token_span': [8, 9], 'label': 'man%1:18:00::'}
-------------------------------------------------------------------------
Preprocessed data:
('hinduism birthday%1:28:00:: man%1:18:00:: call sashti poorthi', '')


In [16]:
# Create the actual datasets 
word_sentences = []
sense_sentences = []
for line in tqdm(lines):
    word_sentence, sense_sentence = preprocess_line(line, nlp, stopwords=stop_words, filtering_regex=regexp_alphbetic, produce_both=True)
    word_sentences.append(word_sentence)
    sense_sentences.append(sense_sentence)

print(f'Preprocessed sentence            : {word_sentences[0]}')
print(f'Preprocessed sentence with senses: {sense_sentences[0]}')

100%|██████████| 534300/534300 [1:01:21<00:00, 145.13it/s]

Preprocessed sentence            : hinduism birthday man call sashti poorthi
Preprocessed sentence with senses: hinduism birthday%1:28:00:: man%1:18:00:: call sashti poorthi


We could filter out uncommon words that have a low frequency but we can do the same specifying the `min_count` parameter of the Word2Vec model

In [17]:
def filter_frequency(sentences, frequency):

    ## remove words that appear only once
    frequency_dict = defaultdict(int)
    for sentence in tqdm(sentences):
        tokens = sentence.split()
        for token in tokens:
            frequency_dict[token] += 1

    texts = [[token for token in sentence.split() if frequency_dict[token] > frequency]
            for sentence in sentences]

    return texts

In [18]:
# ff_word_sentences = filter_frequency(word_sentences, 1)
# ff_sense_sentences = filter_frequency(sense_sentences, 1)

# for i in range(10):
#   print(ff_word_sentences[i])

In [19]:
# Specify the output JSONLines file. Set it to None to skip saving the dataset
output_file = 'preprocessed_dataset.jsonl'
# output_file = None

if output_file is not None:

    # Take all the original lines
    original_sentences = [line['text'] for line in lines]

    # Open the JSONLines file for writing
    with jsonlines.open(output_file, mode='w') as writer:

        for original, preprocessed, preprocessed_with_senses in tqdm(zip(original_sentences, word_sentences, sense_sentences)):

            # Create a dictionary for each set of strings
            data = {
                'original': original,
                'preprocessed': preprocessed,
                'preprocessed_with_senses': preprocessed_with_senses
            }
            
            # Write the dictionary to the JSONLines file
            writer.write(data)

534300it [00:03, 154521.87it/s]


# Word embeddings

1. `Word2Vec` is a word embedding technique that learns vector representations of words, capturing semantic relationships through context.
2. `GloVe` combines global and local word context through matrix factorization to create word embeddings.
3. `FastText` represents words as character n-grams, handling morphological details and out-of-vocabulary words.
4. `Doc2Vec` extends Word2Vec to learn document-level embeddings by treating each document as a unique word.
5. `BERT`, a transformer-based model, learns contextual word embeddings by considering both left and right sentence context.
6. `ELMo` uses bi-directional LSTM to create word embeddings based on entire sentence context, enhancing syntactic and semantic understanding.
7. `USE` is a universal sentence encoder for generating fixed-length embeddings, applicable to various NLP tasks.

More [here](https://medium.com/@vaibhav1403/embedding-techniques-in-natural-language-processing-nlp-29e424ab0cd9).

In [20]:
vector_size = 10
window = 2
min_count = 1 

word_embeddings_model = gensim.models.Word2Vec(word_sentences, vector_size=vector_size, window=window, min_count=min_count)
word_embeddings_model

Load non semantic simlex

In [9]:
simlex_data = wget.download("https://fh295.github.io/SimLex-999.zip")

simple_simlex_path = "data/simlex999/simlex999.tsv"

simplex_pairs = dict()
with zipfile.ZipFile(simlex_data, 'r') as zip, open(simple_simlex_path, "wb") as fw:
   with zip.open('SimLex-999/SimLex-999.txt') as myfile:
    next(myfile)
    for line in myfile:
      print(line)
      w1, w2, pos, score, *_ = line.strip().split()
      w1 = w1.decode('utf-8')
      w2 = w2.decode('utf-8')
      score = float(score)
      simplex_pairs[(w1, w2)] = score
      fw.write(f'{w1}\t{w2}\t{score}\n'.encode('utf-8'))


b'old\tnew\tA\t1.58\t2.72\t2.81\t2\t7.25\t1\t0.41\r\n'
b'smart\tintelligent\tA\t9.2\t1.75\t2.46\t1\t7.11\t1\t0.67\r\n'
b'hard\tdifficult\tA\t8.77\t3.76\t2.21\t2\t5.94\t1\t1.19\r\n'
b'happy\tcheerful\tA\t9.55\t2.56\t2.34\t1\t5.85\t1\t2.18\r\n'
b'hard\teasy\tA\t0.95\t3.76\t2.07\t2\t5.82\t1\t0.93\r\n'
b'fast\trapid\tA\t8.75\t3.32\t3.07\t2\t5.66\t1\t1.68\r\n'
b'happy\tglad\tA\t9.17\t2.56\t2.36\t1\t5.49\t1\t1.59\r\n'
b'short\tlong\tA\t1.23\t3.61\t3.18\t2\t5.36\t1\t1.58\r\n'
b'stupid\tdumb\tA\t9.58\t1.75\t2.36\t1\t5.26\t1\t1.48\r\n'
b'weird\tstrange\tA\t8.93\t1.59\t1.86\t1\t4.26\t1\t1.3\r\n'
b'wide\tnarrow\tA\t1.03\t3.06\t3.04\t2\t4.06\t1\t0.58\r\n'
b'bad\tawful\tA\t8.42\t1.68\t1.92\t1\t3.41\t1\t1.74\r\n'
b'easy\tdifficult\tA\t0.58\t2.07\t2.21\t1\t2.83\t1\t0.36\r\n'
b'bad\tterrible\tA\t7.78\t1.68\t2.04\t1\t2.31\t1\t1.43\r\n'
b'hard\tsimple\tA\t1.38\t3.76\t1.62\t2\t2.16\t1\t1.03\r\n'
b'smart\tdumb\tA\t0.55\t1.75\t2.36\t1\t2.1\t1\t0.47\r\n'
b'insane\tcrazy\tA\t9.57\t1.77\t2.37\t1\t2.09\t1\t0.9

We can search for the best hyperparameters.
- The `vector_size` determines the dimensionality of the word vectors or embeddings.
- The `window_size` defines the context window for the Word2Vec model.
- The `min_count` parameter is used to control the minimum count of a word in the corpus for it to be considered during the training of the Word2Vec model.

In [ ]:

# Define a function to train Word2Vec models and compute a similarity score
def train_word2vec_model(vector_size, window_size, min_count):

    model = Word2Vec(sentences, vector_size=vector_size, window=window_size, min_count=min_count)
    
    # Perform some task using the trained model, e.g., compute a similarity score
    similarity_score = model.wv.similarity("example", "sentence")
    return similarity_score

# Define the hyperparameter grid to search
param_grid = {
    'vector_size': [100, 200, 300],
    'window_size': [3, 5, 7],
    'min_count': [1, 5, 10]
}

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=None, param_grid=param_grid, scoring='neg_mean_squared_error', cv=3)

# Perform the hyperparameter search
grid_search.fit(sentences)

# Get the best hyperparameters and results
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Hyperparameters:", best_params)
print("Best Score:", best_score)


In [ ]:
vector_size = 10
window = 2
min_count = 1 

sense_embeddings_model = gensim.models.Word2Vec(sense_sentences, vector_size=vector_size, window=window, min_count=min_count)
sense_embeddings_model

# Sense embeddings

In [ ]:
# The returned dictionary should be similar to previous word_pair2score 
# but instead of words we consider the senses from the dataset 
# associated with this words
def load_semantic_simplex(path):
  senses2score = dict()
  with open(path) as fr:
    next(fr)
    for line in fr:
      chunks = line.strip().split()
      w1 = chunks[0]
      w2 = chunks[1]
      sim_lex_score = chunks[3]
      senses_w1 = chunks[10].split(",")
      senses_w2 = chunks[11].split(",")
      senses2score[tuple(tuple(senses_w1), tuple(senses_w2))] = sim_lex_score

  return senses2score

In [ ]:

       


def preprocess_text(sentence, stopwords, lemmatize=True, lowercase=True):
  
  doc = nlp(sentence)
  sentence_tokens = []
  
  for token in doc:
  
    token_text = token.lemma_ if lemmatize else token.text
    token_text = token_text.lower() if lowercase else token_text
    
    # Skip stopwords and NON alphanumeric
    if token_text in stopwords or regexp_alphbetic.search(token_text):
      continue
    
    sentence_tokens.append(token_text)
  
  return sentence_tokens


def filter_frequency(sentences, stopwords, frequency, lemmatize=True):

  # Remove common words and tokenize
  texts = [preprocess_text(sentence, stopwords, lemmatize=lemmatize) for sentence in sentences]

  # Remove words that appear only once
  frequency = defaultdict(int)
  for text in texts:
      for token in text:
          frequency[token] += 1

  texts = [[token for token in text if frequency[token] > frequency]
          for text in texts]

  return texts


for i, line in enumerate(lines):
    preprocessed_sentence = preprocess_text(line['text'], stop_words)
    print(preprocessed_sentence)
    if i == 10:
       break
    
cleaned_dataset = filter_frequency(lines, stop_words, 5)

In [ ]:
def decrement_token_spans(text, annotations, stopwords):
    # Tokenize the text into words
    words = text.split()

    # Initialize a variable to keep track of the number of stopwords encountered
    stopwords_count = 0

    # Iterate through annotations and decrement token spans
    for annotation in annotations:
        token_start, token_end = annotation['token_span']

        # Decrement token spans by the number of stopwords before the annotation
        for i in range(token_start, token_end):
            if words[i] in stopwords:
                stopwords_count += 1

        annotation['token_span'][0] -= stopwords_count
        annotation['token_span'][1] -= stopwords_count

    return annotations

# Example usage
text = "In Hinduism , the 60th birthday of a man is called Sashti poorthi ."
annotations = [
    {'token_span': [5, 6], 'label': 'birthday%1:28:00::'},
    {'token_span': [8, 9], 'label': 'man%1:18:00::'}
]
stopwords = ["the", "of", "a", "is", ",", "."]

updated_annotations = decrement_token_spans(text, annotations, stopwords)
print(updated_annotations)

In [ ]:
# Download stopwords
nlp = spacy.load("en_core_web_sm")
regexp_alphbetic = re.compile('[^a-zA-Z]+')

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def remove_stopwords(line, stopwords):

    sentence = line['text']
    tokens = sentence.split()
    
    for annotation in line['annotations']:
        for token in tokens:
            if token in stopwords:
                print(token)

pretty_print(line)
remove_stopwords(line, stop_words)

In [ ]:
# Replace each word with the lemma key in WordNet
def replace_lemma_key(line):

    line_list = line['text'].split()

    for annotation in line['annotations']:

        # Take the lemma key
        lemma_key = annotation['label']

        # Take the position of the lemma key in the sentence
        position = annotation['token_span'][0]
        line_list[position] = lemma_key

    line = ' '.join(line_list)

    return line


line_text_before = line['text']
line_text_after = replace_lemma_key(line)
print(line_text_before)
print(line_text_after)
print(line['text'])

In [ ]:
# Replace each word in the corpus with the respective lemma key
# for line in tqdm(lines):
#     text  = replace_lemma_key(line)
#     line['text'] = text

In [ ]:
def remove_stopwords(sentence, stopwords, lemmatize=True, lowercase=True):
    doc = nlp(sentence)
    sentence_tokens = []

    for token in doc:

        token_text = token.lemma_ if lemmatize else token.text
        token_text = token_text.lower() if lowercase else token_text

        # Skip stopwords and NON alphanumeric
        if token_text in stopwords:
            continue

        sentence_tokens.append(token_text)

    return sentence_tokens


In [ ]:
# Download stopwords
nlp = spacy.load("en_core_web_sm")

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# This regex matches all the WordNet lemma keys 
check_lemma_key = re.compile(r'\w+%\d+:\d+:\d+::')

# This regex matches all the alphanumeric tokens
check_non_alphanumeric = re.compile(r'[^a-zA-Z]+')

# This regex matches non alphanumeric tokens that are not lemma keys
check_non_alphanumeric_and_non_lemma_key = re.compile(r'^[a-zA-Z]+%[0-9]+:[0-9]+:[0-9]+:[a-zA-Z]+:[0-9]+$')

def preprocess_text(sentence, stopwords, lemmatize=True):
  
  doc = nlp(sentence)
  sentence_tokens = []
  
  for token in doc:
  
    token_text = token.lemma_ if lemmatize else token.text
    token_text = token_text.lower()
    
    # Skip stopwords and NON alphanumeric
    if token_text in stopwords or check_non_alphanumeric_and_non_lemma_key.search(token_text):
      continue
    
    sentence_tokens.append(token_text)
  
  return sentence_tokens


text = line['text']
#preprocessed_text = preprocess_text(text, stopwords)
print(text)
#print(preprocessed_text)

for token in text.split():
  res = check_non_alphanumeric_and_non_lemma_key.search(token)
  print(f'{token}: {res}')


In [ ]:
import re

def filter_tokens(sentence):
    # Define regular expression patterns for words and WordNet lemma keys
    word_pattern = r'\b[A-Za-z]+\b'
    lemma_key_pattern = r'\w+%\d+:\d+:\d+::'

    # Create a list of tokens in the original order
    original_tokens = sentence.split()

    filtered_tokens = []
    for token in original_tokens:
        if re.match(word_pattern, token):
            print(f'Token {token} is not valid')
            # filtered_tokens.append(token)
            continue
        elif re.match(lemma_key_pattern, token):
            print(f'Token {token} is a wordnet lemma key')
            filtered_tokens.append(token)
        else:
            print(f'Token {token} is a word')
            filtered_tokens.append(token)

    # Filter tokens while preserving their order
    filtered_tokens = [token for token in original_tokens if re.match(word_pattern, token) or re.match(lemma_key_pattern, token)]

    # Combine the filtered tokens to create the filtered sentence
    filtered_sentence = ' '.join(filtered_tokens)

    return filtered_sentence

# Example usage:
sentence = "This is a sample t::ext with words lif55ke clever and lemma keys like give%2:40:03::."
filtered_result = filter_tokens(sentence)
print(filtered_result)

# Test the function
tokens = [
    "clever%5:00:00:intelligent:00", 
    "give%2:40:03::", 
    "word1%2:40:03::", 
    "123%2:40:03::",
    "wordnet", 
    "WordNet"
]


In [ ]:
def lowercase(text):
    return text.lower()

def remove_stopwords(text, stopwords):
    return text



sentence = 'Io mi chiamo Daniel e faccio schifo'
nlp = spacy.load("en_core_web_sm")
doc = nlp(sentence)
print(doc)

### Create Word2Vec model with Gensim

In [ ]:
vector_size = 10
window = 2
min_count = 1 

word_embeddings_model = gensim.models.Word2Vec(word_dataset, vector_size=vector_size, window=window, min_count=min_count)
word_embeddings_model

In [ ]:
vector_size = 10
window = 2
min_count = 1 

sense_embeddings_model = gensim.models.Word2Vec(sense_dataset, vector_size=vector_size, window=window, min_count=min_count)
sense_embeddings_model

### Load SimLex999 for model evaluation

In [ ]:
import zipfile

simple_simlex_path = "data/simlex999/simlex999.txt"

def load_simlex_from_path(path):
  return None

def get_simlex_from_url(url):
  return None

simlex_pairs = load_simlex_from_path(simple_simlex_path)

### Compute correlation between human scores and Word2vec similarities

In [ ]:
def compute_correlation_score(model, word_pair2score, print_warning=True):
    human_scores = []
    system_scores = []
    for (w1, w2), score in word_pair2score.items():
        if (w1 not in model) or (w2 not in model):
            system_scores.append(-1)
            human_scores.append(score)
            if print_warning:
                print(f"WARNING ({w1} and {w2}) are not present in the embedding model!!" )
        continue
    system_similarity = model.similarity(w1, w2)
    human_scores.append(score)
    system_scores.append(system_similarity)

    human_scores = np.array(human_scores)
    system_scores = np.array(system_scores)
    pearson_r, _ = scipy.stats.pearsonr(human_scores, system_scores)    # Pearson's r
    spearman_rho = scipy.stats.spearmanr(human_scores, system_scores).statistic   # Spearman's rho
    
    return pearson_r, spearman_rho


In [ ]:
word_embeddings_evaluation = compute_correlation_score(model.wv, simplex_pairs, print_warning=False)
sense_embeddings_evaluation = compute_correlation_score(model.wv, simplex_pairs, print_warning=False)
model_pretrained_evaluation = compute_correlation_score(model_pretrained, simplex_pairs)

print(f'Bla Bla Bla')